In [6]:
# Importing libraries

import streamlit as st
import os
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_ollama import OllamaLLM,OllamaEmbeddings

# Load env
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

import warnings
warnings.filterwarnings("ignore")

In [ ]:
## Simple Ollama Call

# Initialize the model
ollama_model_name = "gemma3:4b"
embedding_model = "embeddinggemma:300m"

# Loading the models
import subprocess
subprocess.run(["ollama","pull",ollama_model_name])
# subprocess.run(["ollama","pull",embedding_model])

llm = OllamaLLM(model = ollama_model_name)

llm.invoke("What is capital of germany in a single word")


'Berlin'

In [40]:
# Creating a prompt with history
prompt = ChatPromptTemplate.from_messages([
    ("system","You are smart and expert assistant. Answer the user questions simply"),
    MessagesPlaceholder(variable_name='history'),
    ("human","{question}")
])

# Defining the chain
chain = prompt|llm|StrOutputParser()


In [38]:
from langchain_core.chat_history import BaseChatMessageHistory
# Creating Message History Module
store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_session_history = RunnableWithMessageHistory(chain,get_session_history, input_messages_key='question', history_messages_key='history')

config = {"configurable":{"session_id":"rahul"}}

In [43]:
with_session_history.invoke({"question":"Hi im Rahul"},config=config)

"Hi Rahul! It's great to hear from you again. 😊 \n\nWhat's on your mind today?"

In [44]:
with_session_history.invoke({"question":"what is my name"},config=config)

"Hi Rahul! Your name is Rahul. 😊 \n\nIt's good to confirm that! Is there anything else you’d like to chat about?"

In [45]:
store

{'rahul': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi im Rahul', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Rahul! It's nice to meet you. 😊 \n\nHow can I help you today? Do you have a question you'd like to ask, or were you just saying hello?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi im Rahul', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Rahul! It's great to hear from you again. 😊 \n\nWhat's on your mind today?", additional_kwargs={}, response_metadata={}), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Rahul! Your name is Rahul. 😊 \n\nIt's good to confirm that! Is there anything else you’d like to chat about?", additional_kwargs={}, response_metadata={})])}